Machine Learning - Lab 01 - Feature Engineering

Import Libraries

In [4]:
import pandas as pd
import seaborn as sns
import numpy as np
import xgboost as xg
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn import metrics
from sklearn import tree

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif, f_regression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestClassifier

from imblearn.under_sampling import RandomUnderSampler

In [5]:
LABELS=['label_1','label_2','label_3','label_4']
FEATURES=[f'feature_{i}' for i in range(1,257)]

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
speaker_data_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/CS4622 - Machine Learning/Lab1/train.csv")
speaker_data_train.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_251,feature_252,feature_253,feature_254,feature_255,feature_256,label_1,label_2,label_3,label_4
0,-1.928826,-0.291540,1.105909,2.070652,0.657838,-0.670940,-0.968238,0.049916,1.111149,-1.834592,...,-0.691538,-1.152522,0.441157,-1.494434,-0.344974,-0.395641,45,NaN,1,6
1,-1.709277,-1.008998,0.956611,4.313823,-0.669455,1.928765,0.162550,0.152173,2.356505,-2.898549,...,-0.668235,-0.532695,0.151163,-1.876885,1.651534,-2.706490,45,NaN,1,6
2,-1.906183,-0.357562,-0.682627,4.651838,-0.884940,1.159512,1.354481,-1.432248,2.160144,-3.175735,...,-0.026728,0.440408,2.745906,-0.680754,-0.643588,-2.587203,45,NaN,1,6
3,0.697872,-2.218567,-0.572214,1.192062,0.329554,0.741364,0.962354,0.993512,2.896854,-1.600380,...,0.709032,0.812106,1.055266,-0.915258,-0.924856,-1.318964,45,NaN,1,6
4,-1.976895,-0.496308,0.368102,3.739787,-0.132058,1.234980,-0.501003,0.881554,1.630941,-3.853953,...,-0.599700,-1.024829,1.869753,-2.498570,3.268253,-1.356606,45,NaN,1,6


In [8]:
speaker_data_valid = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/CS4622 - Machine Learning/Lab1/valid.csv")
speaker_data_valid.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_251,feature_252,feature_253,feature_254,feature_255,feature_256,label_1,label_2,label_3,label_4
0,-1.153148,-1.038098,1.419872,2.734152,1.154604,-1.086937,-0.516225,-1.370325,2.865359,-1.879877,...,-0.719046,-1.248530,0.144460,-3.240056,0.052614,0.083108,45,NaN,1,6
1,-1.377524,-1.018393,1.102352,2.849025,0.440302,-1.149039,-0.789796,-2.258196,1.264268,-2.123730,...,-1.383339,-0.787736,1.044895,-2.289637,0.199752,-0.712154,45,NaN,1,6
2,0.889574,-2.743300,-0.320194,3.047766,-0.923335,1.741686,-0.615148,0.756482,2.074775,-1.433126,...,1.178322,0.035333,0.857712,-1.928684,0.639870,-0.268576,45,NaN,1,6
3,-1.527213,-1.133121,0.385927,3.129767,0.229020,1.373105,0.919284,-0.755558,1.086973,-2.440614,...,-0.664176,0.805059,0.975368,-2.700269,1.523236,-1.259052,45,NaN,1,6
4,0.948176,-0.750248,0.008329,1.675338,1.941155,-0.783623,-0.485584,-0.261882,2.875204,-1.473030,...,-0.665200,-0.428060,-0.393100,-1.854657,2.207063,-0.342725,45,NaN,1,6


In [9]:
speaker_data_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/CS4622 - Machine Learning/Lab1/test.csv")
speaker_data_test.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_251,feature_252,feature_253,feature_254,feature_255,feature_256,label_1,label_2,label_3,label_4
0,-1.141206,-0.279703,1.019681,2.605902,0.460391,-1.167380,-1.149165,-1.205164,2.468966,-2.729526,...,-0.868282,-0.429783,0.438561,-1.625122,0.599944,-0.615575,NaN,NaN,NaN,NaN
1,-1.796767,-0.056758,0.771219,4.414086,-1.229059,1.732098,0.063241,-0.126311,1.322355,-3.461282,...,0.227806,0.209200,2.200402,-1.701029,1.163999,-1.570136,NaN,NaN,NaN,NaN
2,-1.388968,-0.418296,0.186543,2.803852,-1.499095,1.395180,0.739648,0.274060,1.228789,-3.081147,...,-0.692679,0.449235,1.377531,-1.854203,-0.040538,-2.164104,NaN,NaN,NaN,NaN
3,1.276844,-0.112323,-0.310796,2.251175,0.283670,-0.581020,-0.024439,-0.306097,2.424685,-1.714863,...,-0.690499,-1.077949,0.930102,-2.758352,-0.246738,-1.389013,NaN,NaN,NaN,NaN
4,-1.560815,-0.168494,1.249770,1.869219,-1.646049,0.132156,-0.664602,0.483327,1.728020,-2.258071,...,-0.510597,1.213396,2.497490,-1.597984,-0.163483,-1.735939,NaN,NaN,NaN,NaN


Standardizing Features

In [10]:
X_train = {}
X_valid = {}
y_train = {}
y_valid = {}

for label in LABELS:
  if (label == 'label_2'):
    speaker_data_train_df = speaker_data_train.dropna()
    speaker_data_valid_df=speaker_data_valid.dropna()

  else:
    speaker_data_train_df = speaker_data_train
    speaker_data_valid_df = speaker_data_valid

  scaler = StandardScaler()
  X_train[label] = pd.DataFrame(scaler.fit_transform(speaker_data_train_df.drop(LABELS,axis=1)),columns=FEATURES)
  y_train[label] = speaker_data_train_df[label]
  X_valid[label] = pd.DataFrame(scaler.transform(speaker_data_valid_df.drop(LABELS,axis=1)),columns=FEATURES)
  y_valid[label] = speaker_data_valid_df[label]

y_train['label_2']

480      25.0
481      25.0
482      25.0
483      25.0
484      25.0
         ... 
28515    29.0
28516    29.0
28517    29.0
28518    29.0
28519    29.0
Name: label_2, Length: 28040, dtype: float64

Label - 01

In [11]:
classifier = SVC(kernel='linear')
classifier.fit(X_train['label_1'],y_train['label_1'])

test_label = speaker_data_test.copy()
test_label = pd.DataFrame(scaler.transform(test_label.drop(LABELS,axis=1)),columns=FEATURES)
y_predict = classifier.predict(test_label)

data_frame_1 = pd.DataFrame(y_predict, columns = ['Predicted labels before feature engineering'])
merged_data_frame = pd.concat([data_frame_1, test_label], axis=0)

In [12]:
principal_component_analysis = PCA(n_components=0.97,svd_solver='full')
principal_component_analysis.fit(X_train['label_1'])
X_train_trf = pd.DataFrame(principal_component_analysis.transform(X_train['label_1']))
X_valid_trf = pd.DataFrame(principal_component_analysis.transform(X_valid['label_1']))
print('Shape after Principal Component Analysis : ', X_train_trf.shape)

classifier.fit(X_train_trf,y_train['label_1'])
y_predict=classifier.predict(principal_component_analysis.transform(X_valid['label_1']))

print("Accuracy : ", metrics.accuracy_score(y_valid['label_1'],y_predict))
print("Precision : ", metrics.precision_score(y_valid['label_1'],y_predict, average='weighted'))
print("Recall : ", metrics.recall_score(y_valid['label_1'],y_predict, average='weighted'))

Shape after Principal Component Analysis :  (28520, 80)
Accuracy :  0.9853333333333333
Precision :  0.9868197746697747
Recall :  0.9853333333333333


In [13]:
transformation = principal_component_analysis.transform(test_label)
transformation = pd.DataFrame(transformation)
columns = [f'new_feature_{i}' for i in range(1,81)]
transformation.columns = columns
print(transformation)

y_predict_test_1 = classifier.predict(principal_component_analysis.transform(test_label))
data_frame_2 = pd.DataFrame(y_predict_test_1, columns=['Predicted labels after feature engineering'])
print(data_frame_2)

data_frame_3 = pd.DataFrame(columns = ['No of new features'], index = data_frame_1.index)
merged2_data_frame = pd.concat([data_frame_1, data_frame_2, data_frame_3, transformation], axis=1)
print(merged2_data_frame)

merged2_data_frame.to_csv('190599B_label_1.csv', index=False)

     new_feature_1  new_feature_2  new_feature_3  new_feature_4  \
0        -1.967567       6.945703      -3.947321      -1.108198   
1        -6.692310       8.884467      -0.489867     -10.240206   
2        -4.810155       3.547653      -0.297522      -5.550987   
3        -2.185614       5.993099      -1.089448      -0.938549   
4         0.695696       6.181181       1.442544      -3.112025   
..             ...            ...            ...            ...   
745       0.370687      -1.536461      -7.516708       4.317870   
746      11.165413       4.127378      -1.536515      -0.896263   
747      -5.593004       2.172272      -2.118896      -1.539570   
748      -4.349980       3.831893       3.496943       3.157099   
749       3.170687       2.672466      -4.072233       1.804944   

     new_feature_5  new_feature_6  new_feature_7  new_feature_8  \
0        -6.440248      -1.053785       3.471608       2.685400   
1        -4.576115       2.229018      -2.025519       2.3664

Label - 02

In [14]:
xgb_regressor = xg.XGBRegressor(objective = 'reg:linear', n_estimators = 10, seed = 123)
xgb_regressor.fit(X_train['label_2'],y_train['label_2'])

y_predict_2 = xgb_regressor.predict(test_label) # Predict the model
data_frame_1_2 = pd.DataFrame(y_predict_2, columns=['Predicted labels before feature engineering'])


[09:23:31] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


In [15]:
selector = SelectKBest(score_func = f_regression, k = 180)
X_train_new_values_L2 = selector.fit_transform(X_train['label_2'],y_train['label_2'])

select_indices = np.where(selector.get_support())[0] # Get the indices of the selected features
select_feature_name = X_train['label_2'].columns[select_indices]
xgb_regressor.fit(X_train_new_values_L2,y_train['label_2'])

[09:23:49] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=10, n_jobs=None, num_parallel_tree=None,
             objective='reg:linear', predictor=None, ...)

In [16]:
y_predict_2_1 = xgb_regressor.predict(selector.transform(test_label))
data_frame_2_2 = pd.DataFrame(y_predict_2_1, columns=['Predicted labels after feature engineering'])

transformation_2 = selector.transform(test_label)
transformation_2 = pd.DataFrame(transformation_2)
columns = [f'new_feature_{i}' for i in range(1,181)]
transformation_2.columns = columns
print(transformation_2)

merged2_data_frame = pd.concat([data_frame_1_2, data_frame_2_2, data_frame_3, transformation_2], axis=1)
merged2_data_frame.to_csv('190599B_label_2.csv', index=False)

     new_feature_1  new_feature_2  new_feature_3  new_feature_4  \
0        -0.924404       0.334858       1.146088       0.427044   
1        -1.586176       0.556571       0.915057      -1.605700   
2        -1.174514       0.197031       0.371399      -1.930607   
3         1.516557       0.501313      -0.091049       0.214414   
4        -1.347989       0.445452       1.360034      -2.107422   
..             ...            ...            ...            ...   
745      -0.297766      -0.611229       0.963826      -0.782036   
746      -0.289396      -1.573867       2.061318      -0.195337   
747       0.389863      -1.014042      -1.078051       0.152771   
748       0.779013       0.356460       2.771686       1.493202   
749      -0.865507      -0.525273       1.429037       0.419843   

     new_feature_5  new_feature_6  new_feature_7  new_feature_8  \
0        -0.771253      -1.004155      -1.844674       1.623322   
1         1.930339       0.280870      -0.822745       0.4040

Label - 03

In [17]:
classifier = RandomForestClassifier(random_state=0)
classifier.fit(X_train['label_3'],y_train['label_3'])

y_predict = classifier.predict(X_valid['label_3'])
print("Accuracy : ", metrics.accuracy_score(y_valid['label_3'],y_predict))
print("Precision : ", metrics.precision_score(y_valid['label_3'],y_predict))
print("Recall : ", metrics.recall_score(y_valid['label_3'],y_predict))

Accuracy :  0.996
Precision :  0.9967159277504105
Recall :  0.9983552631578947


In [18]:
y_predict_3 = classifier.predict(test_label)
data_frame_1_3 = pd.DataFrame(y_predict_3, columns=['Predicted labels before feature engineering'])

In [19]:
principal_component_analysis = PCA(n_components=0.93,svd_solver='full')
principal_component_analysis.fit(X_train['label_3'])

X_train_trf=pd.DataFrame(principal_component_analysis.transform(X_train['label_3']))
X_valid_trf=pd.DataFrame(principal_component_analysis.transform(X_valid['label_3']))
print('Shape after Principal Component Analysis : ', X_train_trf.shape)

classifier.fit(X_train_trf,y_train['label_3'])
y_predict = classifier.predict(principal_component_analysis.transform(X_valid['label_3']))

print("Accuracy : ", metrics.accuracy_score(y_valid['label_3'],y_predict))
print("Precision : ", metrics.precision_score(y_valid['label_3'],y_predict))
print("Recall : ", metrics.recall_score(y_valid['label_3'],y_predict))

Shape after Principal Component Analysis :  (28520, 59)
Accuracy :  0.9986666666666667
Precision :  0.9983579638752053
Recall :  1.0


In [20]:
y_predict = classifier.predict(principal_component_analysis.transform(test_label))
data_frame_2_3 = pd.DataFrame(y_predict, columns=['Predicted labels after feature engineering'])

transformation_3 = principal_component_analysis.transform(test_label)
transformation_3 = pd.DataFrame(transformation_3)
columns = [f'new_feature_{i}' for i in range(1,60)]
transformation_3.columns = columns
transformation_3

merged3_data_frame = pd.concat([data_frame_1_3, data_frame_2_3, data_frame_3, transformation_3], axis=1)
merged3_data_frame.to_csv('190599B_label_3.csv', index=False)

Label - 04

In [21]:
classifier=SVC(kernel='linear',class_weight='balanced')
classifier.fit(X_train['label_4'],y_train['label_4'])

y_predict = classifier.predict(X_valid['label_4'])

print("Accuracy : ", metrics.accuracy_score(y_valid['label_4'],y_predict))
print("Precision : ", metrics.precision_score(y_valid['label_4'],y_predict, average='weighted'))
print("Recall : ", metrics.recall_score(y_valid['label_4'],y_predict, average='weighted'))

Accuracy :  0.9373333333333334
Precision :  0.9493289124787055
Recall :  0.9373333333333334


In [22]:
y_predict_4 = classifier.predict(test_label)
data_frame_1_4 = pd.DataFrame(y_predict_4, columns=['Predicted labels before feature engineering'])

In [23]:
principal_component_analysis = PCA(n_components=0.98,svd_solver='full')
principal_component_analysis.fit(X_train['label_4'])

X_train_trf = pd.DataFrame(principal_component_analysis.transform(X_train['label_4']))
X_valid_trf = pd.DataFrame(principal_component_analysis.transform(X_valid['label_4']))
print('Shape after Principal Component Analysis : ', X_train_trf.shape)

classifier.fit(X_train_trf,y_train['label_4'])
y_predict = classifier.predict(principal_component_analysis.transform(test_label))
data_frame_2_4 = pd.DataFrame(y_predict, columns=['Predicted labels after feature engineering'])

Shape after Principal Component Analysis :  (28520, 89)


In [24]:
transformation_4 = principal_component_analysis.transform(test_label)
transformation_4 = pd.DataFrame(transformation_4)
columns = [f'new_feature_{i}' for i in range(1,90)]
transformation_4.columns = columns
print(transformation_4)

merged4_data_frame = pd.concat([data_frame_1_4, data_frame_2_4, data_frame_3, transformation_4], axis=1)
merged4_data_frame.to_csv('190599B_label_4.csv', index=False)

     new_feature_1  new_feature_2  new_feature_3  new_feature_4  \
0        -1.967567       6.945703      -3.947321      -1.108198   
1        -6.692310       8.884467      -0.489867     -10.240206   
2        -4.810155       3.547653      -0.297522      -5.550987   
3        -2.185614       5.993099      -1.089448      -0.938549   
4         0.695696       6.181181       1.442544      -3.112025   
..             ...            ...            ...            ...   
745       0.370687      -1.536461      -7.516708       4.317870   
746      11.165413       4.127378      -1.536515      -0.896263   
747      -5.593004       2.172272      -2.118896      -1.539570   
748      -4.349980       3.831893       3.496943       3.157099   
749       3.170687       2.672466      -4.072233       1.804944   

     new_feature_5  new_feature_6  new_feature_7  new_feature_8  \
0        -6.440248      -1.053785       3.471608       2.685400   
1        -4.576115       2.229018      -2.025519       2.3664